#### Compute GLM stats across ROIs

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template
# import plotly.io as pio
# pio.renderers.default = "iframe"

# stats
from scipy.stats import permutation_test
def statistic(condA, condB, axis):
    return np.nanmean(condA, axis=axis) - np.nanmean(condB, axis=axis)
n_permutation = 10000

def get_pval(perm_res):
    p_val_unilateral = perm_res.pvalue/2
    p_val_bilateral = perm_res.pvalue

    if p_val_unilateral > 0.05: p_text_unilateral = 'p = {:1.2f}'.format(p_val_unilateral)
    if p_val_unilateral < 0.05: p_text_unilateral = 'p < 0.05'
    if p_val_unilateral < 0.01: p_text_unilateral = 'p < 0.01'
    if p_val_unilateral < 0.001: p_text_unilateral = 'p < 0.001'
    if p_val_unilateral < 0.0001: p_text_unilateral = 'p < 0.0001'

    if p_val_bilateral > 0.05: p_text_bilateral = 'p = {:1.2f}'.format(p_val_bilateral)
    if p_val_bilateral < 0.05: p_text_bilateral = 'p < 0.05'
    if p_val_bilateral < 0.01: p_text_bilateral = 'p < 0.01'
    if p_val_bilateral < 0.001: p_text_bilateral = 'p < 0.001'
    if p_val_bilateral < 0.0001: p_text_bilateral = 'p < 0.0001'
    
    return (p_val_unilateral, p_text_unilateral, p_val_bilateral, p_text_bilateral)


# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['group',
                 'sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']
tasks = ['AttendStimGazeLeft',
         'AttendStimGazeRight',
         'AttendFixGazeLeft',
         'AttendFixGazeRight']
hemis = ['L', 'R']
cortical_mask = 'cortical'
best_voxels_num = 250

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)
group_tsv_dir = '{}/{}/glm/tsv'.format(pp_dir, 'group')

In [ ]:
# load results
first_df = 1
for subject_num, subject in enumerate(subjects): 
    fit_dir = '{}/{}/glm/fit'.format(pp_dir, subject)
    mask_dir = '{}/{}/masks'.format(pp_dir, subject)
    prf_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
    th_mat = nb.load('{}/{}_task-FullScreen_prf_threshold.nii.gz'.format(mask_dir,subject)).get_fdata()
    r2_th_mat = nb.load('{}/{}_task-FullScreen_par-r2.nii.gz'.format(prf_dir,subject)).get_fdata()*th_mat

    for task in tasks:
        # load data
        beta_mat = nb.load('{}/{}_task-{}_beta.nii.gz'.format(fit_dir, subject, task)).get_fdata()*th_mat

        print('computing {} {}'.format(subject, task))
        for roi_num, roi in enumerate(rois):

            # load roi
            lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
            rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()

            # select data by roi mask
            r2_roi_th_lh_mat = r2_th_mat[lh_mat==True]
            r2_roi_th_rh_mat = r2_th_mat[rh_mat==True]
            beta_roi_th_lh_mat = beta_mat[lh_mat==True]
            beta_roi_th_rh_mat = beta_mat[rh_mat==True]

            df_lh_roi = pd.DataFrame({'subject': [subject] * r2_roi_th_lh_mat.shape[0],
                                      'roi': [roi] * r2_roi_th_lh_mat.shape[0],
                                      'task': [task] * r2_roi_th_lh_mat.shape[0],
                                      'r2': r2_roi_th_lh_mat,
                                      'beta': beta_roi_th_lh_mat})
            df_lh_roi['rank_fs_r2'] = df_lh_roi.groupby('roi')['r2'].rank(method='max', ascending=False)
            df_lh_best_roi = df_lh_roi[(df_lh_roi.rank_fs_r2<=best_voxels_num)]


            df_rh_roi = pd.DataFrame({'subject': [subject] * r2_roi_th_rh_mat.shape[0],
                                      'roi': [roi] * r2_roi_th_rh_mat.shape[0],
                                      'task': [task] * r2_roi_th_rh_mat.shape[0],
                                      'r2': r2_roi_th_rh_mat,
                                      'beta': beta_roi_th_rh_mat})
            df_rh_roi['rank_fs_r2'] = df_rh_roi.groupby('roi')['r2'].rank(method='max', ascending=False)
            df_rh_best_roi = df_rh_roi[(df_rh_roi.rank_fs_r2<=best_voxels_num)]

            
            df_roi = pd.DataFrame({'subject': [subject],
                                   'roi': [roi],
                                   'task': [task],
                                   'beta_lh_median' : df_lh_best_roi.beta.median(),
                                   'beta_rh_median' : df_rh_best_roi.beta.median(),
                                  })
            
            # across roi
            if first_df: 
                df = df_roi
                first_df = 0
            else:
                df = pd.concat([df,df_roi], ignore_index=True)     

df.roi = pd.Categorical(df.roi,categories=rois)

In [ ]:
# group data
df_group = pd.DataFrame({'subject': ['group']*len(rois)*len(tasks)})
df_group['roi'] = rois*len(tasks)
df_group['task'] = [tasks[0]]*len(rois) + [tasks[1]]*len(rois) + [tasks[2]]*len(rois) + [tasks[3]]*len(rois)
df_group['beta_lh_median_avg']=df.groupby(['task','roi']).mean().beta_lh_median.reset_index(name='beta_lh_median_avg').beta_lh_median_avg
df_group['beta_lh_median_sem']=df.groupby(['task','roi']).sem(ddof=-1).beta_lh_median.reset_index(name='beta_lh_median_sem').beta_lh_median_sem
df_group['beta_rh_median_avg']=df.groupby(['task','roi']).mean().beta_rh_median.reset_index(name='beta_rh_median_avg').beta_rh_median_avg
df_group['beta_rh_median_sem']=df.groupby(['task','roi']).sem(ddof=-1).beta_rh_median.reset_index(name='beta_rh_median_sem').beta_rh_median_sem

In [ ]:
# Subplot settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)

fig_template = plotly_template(template_specs)
rows, cols = 3, 4
fig_height, fig_width = 1000,1000
column_widths,row_heights = [1,1,1,1],[1,1,1]
sb_specs = [[{},{},{},{}],[{},{},{},{}],[{},{},{},{}]]
hover = 'Beta: %{y:1.2f}'
xaxis_range = [0,1]
xaxis_tickvals = [0.25,0.75]
xaxis_ticktext = ['Left','Right']
lh_col = "rgba(0, 0, 0, 1)"
rh_col = "rgba(157, 157, 157, 1)"
lh_area_col = "rgba(0, 0, 0, 0.2)"
rh_area_col = "rgba(157, 157, 157, 0.2)"

yaxis_range = [0,2]
attend_tasks = ['attend-bar', 'attend-fix']
attend_tasks_data = ['AttendStim', 'AttendFix']


for subject_num, subject in enumerate(subjects_plot):
    tsv_dir = '{}/{}/glm/'.format(pp_dir, subject)
    os.makedirs(tsv_dir,exist_ok=True) 
    
    for attend_task, attend_task_data in zip(attend_tasks, attend_tasks_data):
        subplot_titles = ['<b>V1</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>V2</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>V3</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>V3AB</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>hMT+</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>LO</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>VO</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>iIPS</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>sIPS</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>iPCS</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>sPCS</b><br> ({}, <i>{}</i>)'.format(subject, attend_task), 
                          '<b>mPCS</b><br> ({}, <i>{}</i>)'.format(subject, attend_task)]

        fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                            column_widths=column_widths, row_heights=row_heights, subplot_titles=subplot_titles)
    
        for roi_num, roi in enumerate(rois):
            # plot data
            row_num = np.ceil((roi_num+1)/cols)
            if roi_num == 0 or roi_num == 4 or roi_num == 8: col_num = 1
            elif roi_num == 1 or  roi_num == 5 or roi_num == 9: col_num = 2
            elif roi_num == 2 or  roi_num == 6 or roi_num == 10: col_num = 3
            elif roi_num == 3 or  roi_num == 7 or roi_num == 11: col_num = 4
    
            if roi_num == 0 :showlegend=True
            else: showlegend=False
            
            # beta 1 line
            fig.add_shape(type="line", x0=xaxis_range[0], y0=1, x1=xaxis_range[1], y1=1, 
                          line_color='black', line_dash="dot", line_width=1, row=int(row_num), col=int(col_num))

            # define data
            if subject == 'group':
                lh_val = [df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeLeft'.format(attend_task_data))].beta_lh_median_avg.item(),
                          df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeRight'.format(attend_task_data))].beta_lh_median_avg.item()]
                rh_val = [df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeLeft'.format(attend_task_data))].beta_rh_median_avg.item(),
                          df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeRight'.format(attend_task_data))].beta_rh_median_avg.item()]
                
                lh_sem = [df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeLeft'.format(attend_task_data))].beta_lh_median_sem.item(),
                          df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeRight'.format(attend_task_data))].beta_lh_median_sem.item()]            
                rh_sem = [df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeLeft'.format(attend_task_data))].beta_rh_median_sem.item(),
                          df_group.loc[(df_group.subject==subject) & (df_group.roi==roi) & (df_group.task=='{}GazeRight'.format(attend_task_data))].beta_rh_median_sem.item()]
                
            else:
                lh_val = [df.loc[(df.subject==subject) & (df.roi==roi) & (df.task=='{}GazeLeft'.format(attend_task_data))].beta_lh_median.item(),
                          df.loc[(df.subject==subject) & (df.roi==roi) & (df.task=='{}GazeRight'.format(attend_task_data))].beta_lh_median.item()]
                rh_val = [df.loc[(df.subject==subject) & (df.roi==roi) & (df.task=='{}GazeLeft'.format(attend_task_data))].beta_rh_median.item(),
                          df.loc[(df.subject==subject) & (df.roi==roi) & (df.task=='{}GazeRight'.format(attend_task_data))].beta_rh_median.item()]
            
                
            # scatter plot           
            # left hemi
            fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(lh_val), 
                                        legendgroup='lh', name='<i>left hemi.<i>',
                                        showlegend=showlegend, mode='markers+lines', marker_color=lh_col, hovertemplate=hover,
                                        line_width=1, opacity=1, marker_size=6),row=int(row_num), col=int(col_num))
            if subject == 'group':
                fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(lh_val)+np.array(lh_sem),
                                            mode='lines',line_width=0, fillcolor=lh_area_col, showlegend=False, legendgroup='lh', 
                                            hoverinfo='skip'),row=int(row_num), col=int(col_num))
                fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(lh_val)-np.array(lh_sem),
                                            mode='lines',line_width=0, fillcolor=lh_area_col, showlegend=False, legendgroup='lh', 
                                            hoverinfo='skip',fill='tonexty',), row=int(row_num), col=int(col_num))
    
            # right hemi
            fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(rh_val), 
                                        legendgroup='rh', name='<i>right hemi.<i>',
                                        showlegend=showlegend, mode='markers+lines', marker_color=rh_col, hovertemplate=hover,
                                        line_width=1, opacity=1, marker_size=6),row=int(row_num), col=int(col_num))
            if subject == 'group':
                fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(rh_val)+np.array(rh_sem),
                                            mode='lines',line_width=0, fillcolor=rh_area_col, showlegend=False, legendgroup='rh', 
                                            hoverinfo='skip'),row=int(row_num), col=int(col_num))
                fig.append_trace(go.Scatter(x=xaxis_tickvals, y=np.array(rh_val)-np.array(rh_sem),
                                            mode='lines',line_width=0, fillcolor=rh_area_col, showlegend=False, legendgroup='rh', 
                                            hoverinfo='skip',fill='tonexty',), row=int(row_num), col=int(col_num))

        # set axis
        for row in np.arange(rows):
            for col in np.arange(cols):
                fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
                fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)

        # set figure 
        for xaxis in ['xaxis', 'xaxis2', 'xaxis3', 'xaxis4', 'xaxis5', 'xaxis6', 
                      'xaxis7', 'xaxis8', 'xaxis9','xaxis10', 'xaxis11', 'xaxis12']:
            exec("fig.layout.update({}_range = xaxis_range)".format(xaxis))
            exec("fig.layout.update({}_tickvals = xaxis_tickvals)".format(xaxis))
            exec("fig.layout.update({}_ticktext = xaxis_ticktext)".format(xaxis))
            exec("fig.layout.update({}_title = 'Gaze direction')".format(xaxis)),
    
        for yaxis in ['yaxis', 'yaxis2', 'yaxis3', 'yaxis4', 'yaxis5', 'yaxis6', 
                      'yaxis7', 'yaxis8', 'yaxis9', 'yaxis10', 'yaxis11', 'yaxis12']:
            exec("fig.layout.update({}_range = yaxis_range)".format(yaxis))
            if yaxis in ['yaxis', 'yaxis5', 'yaxis9']:
                exec("fig.layout.update({}_title = 'GLM Beta (a.u)')".format(yaxis)),
        
        fig.layout.update(template=fig_template, width=fig_width, height=fig_height, margin_l=60, margin_r=20, margin_t=50, margin_b=70,
                          legend_yanchor='top', legend_y=0.99, legend_xanchor='left', legend_x=0.1, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=0.05)
    
        # show and save figure
        fig.show(config={"displayModeBar": False})
        fig.write_image("{}/{}_glm_cmp_{}.pdf".format(tsv_dir, subject, attend_task_data))
        fig.write_html("{}/{}_glm_cmp_{}.html".format(tsv_dir, subject, attend_task_data),config={"displayModeBar": False})    